In [1]:
import numpy as np
import os
# Required imports for models and training
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from torch.utils.data import DataLoader
import torch
from torch import nn
import torch.nn.functional as F
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import os

c:\Users\Akshat\anaconda3\envs\cuda_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Code to download the dataset from huggingface

In [2]:
'''Reference: https://huggingface.co/datasets/rajpurkar/squad_v2'''
# from datasets import load_dataset

# ds = load_dataset("rajpurkar/squad_v2")
# ds.save_to_disk("squad_v2")

'Reference: https://huggingface.co/datasets/rajpurkar/squad_v2'

In [3]:
#load the presaved dataset
from datasets import load_from_disk
squad_v2 = load_from_disk("squad_v2")

Understanding the dataset

In [4]:
print(squad_v2)

#printing the questions and answers for the same context
train_dataset = squad_v2["train"]
val_dataset = squad_v2["validation"]
print(train_dataset[0])

contexts = set()
for i in range(len(train_dataset)):
    contexts.add(train_dataset[i]["context"])
print(len(contexts))

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})
{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "C

In [ ]:
def exact_match_score(predictions, references):
    assert len(predictions) == len(references), "Lists must have the same length"
    matches = sum(p == r for p, r in zip(predictions, references))
    return matches / len(references) * 100  # Convert to percentage

from multiprocessing import cpu_count

def preprocess_data(examples, tokenizer, max_length=384):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]
    
    # Tokenize inputs
    tokenized = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    # Get the sample mapping
    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized.pop("offset_mapping")
    
    # Get answer positions
    start_positions = []
    end_positions = []
    
    for i, offset in enumerate(offset_mapping):
        # Get the sample index from the mapping
        sample_idx = sample_mapping[i]
        
        # Get the answer for this sample
        answer = examples["answers"][sample_idx]
        start_char = answer["answer_start"][0] if answer["text"] else -1
        end_char = start_char + len(answer["text"][0]) if answer["text"] else -1
        
        # Get sequence IDs to find context tokens
        sequence_ids = tokenized.sequence_ids(i)
        
        # Find context boundaries
        idx = 0
        while idx < len(sequence_ids) and sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        
        while idx < len(sequence_ids) and sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
        
        # Set answer positions
        if start_char == -1 or end_char == -1:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and not (offset[idx][0] <= start_char < offset[idx][1]):
                idx += 1
            start_positions.append(idx if idx <= context_end else 0)
            
            idx = context_end
            while idx >= context_start and not (offset[idx][0] <= end_char <= offset[idx][1]):
                idx -= 1
            end_positions.append(idx if idx >= context_start else 0)
    
    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions
    return tokenized

# Model definitions
class SpanBERTModel(nn.Module):
    def __init__(self, model_name="SpanBERT/spanbert-base-cased"):
        super().__init__()
        self.bert = AutoModelForQuestionAnswering.from_pretrained(model_name)
        
    def forward(self, input_ids, attention_mask, start_positions=None, end_positions=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            start_positions=start_positions,
            end_positions=end_positions
        )
        return outputs

# Initialize models and tokenizer
tokenizer = AutoTokenizer.from_pretrained("SpanBERT/spanbert-base-cased")

# Training configuration
config = {
    "batch_size": 8,
    "learning_rate": 2e-2,
    "num_epochs": 10,
    "max_length": 256,
    "train_size": 15000  # As per requirement to use at least 15000 samples
}

# Prepare datasets with optimized processing
print("Preprocessing training data...")
train_subset = train_dataset.select(range(config["train_size"]))

# Prepare datasets with smaller batch for testing
print("Preprocessing training data...")
train_subset = train_dataset.select(range(config["train_size"]))
train_encoded = train_subset.map(
    lambda x: preprocess_data(x, tokenizer, config["max_length"]),
    batched=True,
    remove_columns=train_subset.column_names,
    batch_size=32  # Process in smaller batches
)

print("Preprocessing validation data...")
val_encoded = val_dataset.map(
    lambda x: preprocess_data(x, tokenizer, config["max_length"]),
    batched=True,
    remove_columns=val_dataset.column_names,
    batch_size=32  # Process in smaller batches
)

# Save the encoded data
save_dir = "encoded_data"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

print("Saving encoded datasets...")
train_encoded.save_to_disk(os.path.join(save_dir, "train_encoded"))
val_encoded.save_to_disk(os.path.join(save_dir, "val_encoded"))

print("Done!")

Done!


In [ ]:
from datasets import load_from_disk
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import os
import numpy as np

class QADataset(Dataset):
    def __init__(self, encoded_dataset, max_length=384):
        self.encoded_dataset = encoded_dataset
        self.max_length = max_length
        
    def __len__(self):
        return len(self.encoded_dataset)
    
    def __getitem__(self, idx):
        item = self.encoded_dataset[idx]
        
        # Ensure all sequences are of the same length
        input_ids = item['input_ids'][:self.max_length]
        attention_mask = item['attention_mask'][:self.max_length]
        
        # Pad sequences if necessary
        if len(input_ids) < self.max_length:
            padding_length = self.max_length - len(input_ids)
            input_ids = input_ids + [0] * padding_length
            attention_mask = attention_mask + [0] * padding_length
        
        # Convert to tensors
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'start_positions': torch.tensor(item['start_positions'], dtype=torch.long),
            'end_positions': torch.tensor(item['end_positions'], dtype=torch.long)
        }

def collate_fn(batch):
    """Custom collate function to handle variable length sequences"""
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    start_positions = torch.stack([item['start_positions'] for item in batch])
    end_positions = torch.stack([item['end_positions'] for item in batch])
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'start_positions': start_positions,
        'end_positions': end_positions
    }

# Load encoded datasets
save_dir = "encoded_data"
train_encoded = load_from_disk(os.path.join(save_dir, "train_encoded"))
val_encoded = load_from_disk(os.path.join(save_dir, "val_encoded"))

# Convert to PyTorch datasets
train_dataset = QADataset(train_encoded, max_length=config["max_length"])
val_dataset = QADataset(val_encoded, max_length=config["max_length"])

# Create dataloaders with custom collate function
train_dataloader = DataLoader(
    train_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    pin_memory=True,
    collate_fn=collate_fn
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=config["batch_size"],
    pin_memory=True,
    collate_fn=collate_fn
)

def train_model(model, train_dataloader, val_dataloader, config):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config["learning_rate"])
    
    train_losses = []
    val_losses = []
    em_scores = []
    
    for epoch in range(config["num_epochs"]):
        # Training phase
        model.train()
        total_train_loss = 0
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
        
        for batch in progress_bar:
            try:
                # Move batch to device
                batch = {k: v.to(device) for k, v in batch.items()}
                
                # Forward pass
                outputs = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    start_positions=batch["start_positions"],
                    end_positions=batch["end_positions"]
                )
                
                loss = outputs["loss"] if isinstance(outputs, dict) else outputs.loss
                total_train_loss += loss.item()
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # Update progress bar
                progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
                
            except RuntimeError as e:
                print(f"Error in batch: {e}")
                continue
        
        avg_train_loss = total_train_loss / len(train_dataloader)
        train_losses.append(avg_train_loss)
        
        # Validation phase
        model.eval()
        total_val_loss = 0
        predictions = []
        references = []
        
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validation"):
                try:
                    batch = {k: v.to(device) for k, v in batch.items()}
                    
                    outputs = model(
                        input_ids=batch["input_ids"],
                        attention_mask=batch["attention_mask"],
                        start_positions=batch["start_positions"],
                        end_positions=batch["end_positions"]
                    )
                    
                    loss = outputs["loss"] if isinstance(outputs, dict) else outputs.loss
                    total_val_loss += loss.item()
                    
                    # Get predictions
                    start_logits = outputs.start_logits
                    end_logits = outputs.end_logits
                    
                    start_pred = torch.argmax(start_logits, dim=1)
                    end_pred = torch.argmax(end_logits, dim=1)
                    
                    # Convert predictions to text
                    for i in range(len(start_pred)):
                        pred_text = tokenizer.decode(
                            batch["input_ids"][i][start_pred[i]:end_pred[i]+1],
                            skip_special_tokens=True
                        )
                        ref_text = tokenizer.decode(
                            batch["input_ids"][i][batch["start_positions"][i]:batch["end_positions"][i]+1],
                            skip_special_tokens=True
                        )
                        predictions.append(pred_text)
                        references.append(ref_text)
                        
                except RuntimeError as e:
                    print(f"Error in validation batch: {e}")
                    continue
        
        avg_val_loss = total_val_loss / len(val_dataloader)
        val_losses.append(avg_val_loss)
        
        # Calculate exact match score
        em_score = exact_match_score(predictions, references)
        em_scores.append(em_score)
        
        print(f"\nEpoch {epoch+1}:")
        print(f"Average training loss: {avg_train_loss:.4f}")
        print(f"Average validation loss: {avg_val_loss:.4f}")
        print(f"Exact match score: {em_score:.2f}%")
        print("-" * 50) 
    
    return train_losses, val_losses, em_scores
# Train SpanBERT model
print("Training SpanBERT model...")
spanbert_model = SpanBERTModel()
train_losses, val_losses, em_scores = train_model(spanbert_model, train_dataloader, val_dataloader, config)

# Plot results
plt.figure(figsize=(15, 5))

# Plot losses
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot EM scores
plt.subplot(1, 2, 2)
plt.plot(em_scores, label='EM Score', color='green')
plt.title('Exact Match Score')
plt.xlabel('Epoch')
plt.ylabel('Score (%)')
plt.legend()

plt.tight_layout()
plt.show()

Training SpanBERT model...


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cuda


Validation: 100%|██████████| 1538/1538 [02:27<00:00, 10.40it/s]



Epoch 1:
Average training loss: 5.9808
Average validation loss: 5.9506
Exact match score: 51.46%
--------------------------------------------------


Epoch 2:   6%|▌         | 116/1912 [00:37<10:00,  2.99it/s, loss=5.9353]